## 오차역전파(Backpropagation)

* 이전 단원에서는 가중치 매개변수의 기울기는 수치 미분을 사용해 구했다.        
* 수치 미분은 단순하고, 구현하기 쉽지만 계산 시간이 오래 걸린다.        
* 따라서, 이 가중치 매개변에 대한 손실함수의 기울기를 효율적으로 계산하는 오차 역전파법을 배워본다.

### 5.1 계산 그래프      

* 계산 그래프 : 계산 과정을 그래프로 나타낸 것, 복수의 노드와 에지로 나타냄.

> Q1. 현빈 군은 슈퍼에서 1개에 100원인 사과를 2개 샀습니다. 이 때 지불 금액을 구하세요. 단 소비세가 10% 부과됩니다.         
> 사과  -----100-----(*2)----> 200----->(*1.1)------> 지불

* 계산 그래프를 이용한 문제풀이        
1. 계산 그래프를 구성한다.       
2. 그래프 계산을 왼쪽에서 오른쪽으로 진행한다.(__순전파__ )          
__오차 '역전파'__ 법에서 알 수 있듯이, __오른쪽 -> 왼쪽으로 흐르는 계산법을 역전파___ 라고 한다.

#### 5.1.2 국소적 계산       

* 자신과 직접 관계된 작은 범위        
* 전체 계산이 아무리 복잡하더라도 각 단계에서 하는 일은 해당 노드의 '국소적 계산'   

####  5.1.3 왜 국소적 계산으로 푸는가?      

* 국소적 계산을 이용하여, 전체가 아무리 복잡해도 각 노드에서는 단순한 계산에 집중하여 문제를 단순화 할 수 있음      
* 계산 그래프는 "중간 계산 결과를 저장할 수 있음"       
* __역전파를 통해 미분을 효율적으로 계산할 수 있음__ 

> 역전파는 굵은 화살표로 그림
> ![오차역전파법](./PostingPic/오차역전파법.png)

* 국소적 미분을 전달하고, 그 미분 값은 화살표 아래에 적는다.     

#### 5.2 연쇄법칙        

* 합성함수 : 여러 함수로 구성된 함수.      
> $z= t^2$ , $t = x+y$

* 연쇄 법칙 : 합성 함수의 미분에 대한 성질          
> 합성 함수의 미분은 합성 함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수 있다.

#### 5.3 역전파

#### 5.3.1 덧셈 노드의 역전파       
* 1을 곱하기만 한다(입력된 값을 그대로 다음 노드로 흐르게 됨)   

#### 5.3.2 곱셈 노드의 역전파        
* 상류의 값 * 순전파의 입력 신호를 서로 바꾼 값 

#### 5.3.3 사과 쇼핑의 예         

#### 5.4 단순한 계층 구현하기      

#### 5.4.1 곱셈 계층        
* 모든 계층은 forward() 와 backward() 라는 공통의 메서드를 갖도록 구현한다.     
* forward() : 순전파        
* backward() : 역전파          

In [1]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x*y
        
        return out
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        
        return dx, dy

In [6]:
apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price,tax)

print(price)

220.00000000000003


In [7]:
dprice = 1

dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


In [8]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x+y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

In [10]:
# 사과 2개 사기
# 오렌지 3개 사기
apple = 100
apple_num = 2

orange = 150
orange_num = 3

tax = 1.1

# 사과 2개를 곱하는 레이어
mul_apple_layer = MulLayer()

# 오렌지 3개를 곱하는 레이어
mul_orange_layer = MulLayer()

# 구매한 과일들의 값을 곱하는 레이어
add_apple_orange_layer = AddLayer()

# 소비세를 곱하는 레이어
mul_tax_layer = MulLayer()


apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)

all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

print("총 가격 출력 : ", price)

총 가격 출력 :  715.0000000000001


In [11]:
deprice = 1

de_all_price, de_tax = mul_tax_layer.backward(deprice)

de_apple_price, de_orange_price = add_apple_orange_layer.backward(de_all_price)

de_orange, de_orange_num = mul_orange_layer.backward(de_orange_price)
de_apple, de_apple_num = mul_apple_layer.backward(de_apple_price)

print("de apple num, de apple, de orange num, de orange")
print(de_apple_num, de_apple, de_orange_num, de_orange)

de apple num, de apple, de orange num, de orange
110.00000000000001 2.2 165.0 3.3000000000000003


## 활성화 함수 구현하기        

#### 5.5.1 ReLU 함수 구현하기          

x>0 일 때는 1, x <=0 이면 0

In [12]:
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        
        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

* ReLU 층은 mask라는 넘파이 배열 변수를 갖는다. 
* mask는 True False로 이루어져 있다.

In [13]:
import numpy as np

x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

mask = (x <= 0)
print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


## 5.5.2 Sigmoid 계층 구현하기

--- 
- 추가된 exp 노드는 지수(exp) 계산    
- / 노드는 나누기 계산을 수행한다.

In [1]:
class sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1/(1+np.exp(-x))
        self.out = out
        
        return out
    
    def backward(self, dout):
        dx = dout *(1.0 - self.out) * self.out
        
        return dx

* 신경망의 순전파에서 수행하는 행렬의 곱은 기하학에서는 '어파인 변환' 이라고 함,